# Estimating how active actors will be

In [1]:
! wget "https://datasets.imdbws.com/title.principals.tsv.gz"
! wget "https://datasets.imdbws.com/title.basics.tsv.gz"
! wget "https://datasets.imdbws.com/name.basics.tsv.gz"

--2019-03-29 03:22:20--  https://datasets.imdbws.com/title.principals.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.239.4, 99.84.239.107, 99.84.239.96, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.239.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279868836 (267M) [binary/octet-stream]
Saving to: ‘title.principals.tsv.gz’

title.principals.ts 100%[===================>] 266.90M  13.9MB/s    in 22s     

2019-03-29 03:22:42 (12.1 MB/s) - ‘title.principals.tsv.gz’ saved [279868836/279868836]

--2019-03-29 03:22:43--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.239.4, 99.84.239.107, 99.84.239.96, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.239.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103032984 (98M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>

In [2]:
! gunzip title.principals.tsv.gz
! gunzip title.basics.tsv.gz
! gunzip name.basics.tsv.gz

gzip: title.principals.tsv already exists; do you wish to overwrite (y or n)? n
	not overwritten
gzip: title.basics.tsv already exists; do you wish to overwrite (y or n)? n
	not overwritten
gzip: name.basics.tsv already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [0]:
import pandas as pd
import numpy as np

In [4]:
cast = pd.read_csv('title.principals.tsv', sep='\t', na_values = "\\N")
titles = pd.read_csv('title.basics.tsv', sep='\t', na_values = "\\N")
names = pd.read_csv('name.basics.tsv', sep='\t', na_values = "\\N")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
cast.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Herself""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [6]:
cast.dtypes

tconst        object
ordering       int64
nconst        object
category      object
job           object
characters    object
dtype: object

In [7]:
type(cast.tconst)

pandas.core.series.Series

In [8]:
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,NaN,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"Comedy,Short"


In [9]:
names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0072308,tt0043044,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,producer","tt0054452,tt0059956,tt0057345,tt0049189"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,soundtrack","tt0078723,tt0080455,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0083922,tt0069467,tt0050986,tt0050976"


In [0]:
actors = ["Lauren Bacall", "Cary Grant"]

In [0]:
names_actors = names[names.primaryName.isin(actors)].head(len(actors))

In [12]:
names_actors

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
25,nm0000026,Cary Grant,1904.0,1986.0,"actor,soundtrack,producer","tt0056923,tt0036613,tt0048728,tt0053125"


In [0]:
cast_actors = pd.merge(names_actors, cast, on = 'nconst')

In [0]:
movie_actors = pd.merge(cast_actors, titles, on = 'tconst')

In [0]:
movie_list = movie_actors[['primaryName','birthYear','deathYear','category','primaryTitle','startYear']]

In [0]:
movie_list = movie_list[movie_list.category.isin(['actor','actress'])]

In [17]:
movie_list

,primaryName,birthYear,deathYear,category,primaryTitle,startYear
0,Lauren Bacall,1924.0,2014.0,actress,To Have and Have Not,1944.0
1,Lauren Bacall,1924.0,2014.0,actress,Confidential Agent,1945.0
2,Lauren Bacall,1924.0,2014.0,actress,The Big Sleep,1946.0
3,Lauren Bacall,1924.0,2014.0,actress,Dark Passage,1947.0
4,Lauren Bacall,1924.0,2014.0,actress,Key Largo,1948.0
5,Lauren Bacall,1924.0,2014.0,actress,Bright Leaf,1950.0
6,Lauren Bacall,1924.0,2014.0,actress,Young Man with a Horn,1950.0
7,Lauren Bacall,1924.0,2014.0,actress,How to Marry a Millionaire,1953.0
8,Lauren Bacall,1924.0,2014.0,actress,Woman's World,1954.0
9,Lauren Bacall,1924.0,2014.0,actress,Blood Alley,1955.0


In [0]:
movie_list.loc["startYear"] = movie_list.startYear.astype(int) 

In [0]:
startYear = 1950
endYear = 1960

In [0]:
movie_list_observed = movie_list[movie_list.startYear.isin(range(startYear,endYear))]
movie_list_unobserved = movie_list[movie_list.startYear >= endYear]

In [0]:
movie_counts = movie_list_observed.groupby(['primaryName','startYear'])['primaryTitle'].count().reset_index()

In [22]:
movie_counts.groupby('primaryName')['primaryTitle'].sum()

primaryName
Cary Grant       13
Lauren Bacall    13
Name: primaryTitle, dtype: int64